In [1]:
"""
Generate descriptive SNBB figures that match the slide template colours.
--------------------------------------------------------------------------

Prereqs:
    pip install matplotlib pandas numpy seaborn
    (seaborn only used for its convenient styling helper)

Replace CSV_PATH if your file lives elsewhere.
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from matplotlib import font_manager as fm

In [2]:
SOFTWARE = "mrtrix3"
ATLAS = "schaefer2018tian2020_400_7"
RECONSTRUCTION = "SDStream"
SCALE = "None"
WEIGHT = "SIFT2"


DISTRIBUTION_METRIC = "qfmean"
DIFFUSION_MEASURES = ["adc", "fa", "rd", "ad"]
STRUCTURAL_MEASURES = ["gm_vol", "wm_vol", "csf_vol"]
CONNECTOME_MEASURES = [
    "degree",
    "eigenvector_centrality",
    "betweenness_centrality",
    "degree_norm",
    "regional_efficiency",
]

In [3]:
sessions = pd.read_csv("~/Projects/plasticityhub/sessions.csv")
sessions["subject_code"] = (
    sessions["subject_code"]
    .astype(str)
    .str.replace("-", "")
    .str.replace(" ", "")
    .str.replace("_", "")
    .str.zfill(4)
)
sessions.loc[sessions["filled"], "weekly_workouts"] = sessions.loc[sessions["filled"], "weekly_workouts"].fillna(0)

# sessions.loc[sessions["filled"], "weekly_workouts"].fillna(0, inplace=True)
# sessions["weekly_workouts"] = sessions["weekly_workouts"].fillna(0)
sessions["weekly_workouts"] = sessions["weekly_workouts"].replace({"10 שעות ויותר": 10})
sessions.loc[sessions["weekly_workouts"].notna(), "weekly_workouts"] = sessions.loc[
    sessions["weekly_workouts"].notna(), "weekly_workouts"
].astype(int)
sessions.loc[~sessions["filled"], "weekly_workouts"] = np.nan

In [4]:
sessions["weekly_workouts"].quantile([0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 1.0])

0.25     0.0
0.50     0.0
0.75     2.0
0.90     4.0
0.95     5.0
0.99     7.0
1.00    14.0
Name: weekly_workouts, dtype: object

In [5]:
MAX_NUMBER_OF_WORKOUTS = 0

# write a query to detect atheletes (study == "sport", condition != "learning")
ATHLETES_QUERY = "(condition in ['Professional', 'Control']) & (group in ['Climbing', 'Bjj'])"
LEARNERS_QUERY = "(condition in ['Learning']) & (group in ['Climbing', 'Bjj'])"

target_subjects = sessions.query(ATHLETES_QUERY)["subject_code"].unique()
learner_subjects = sessions.query(LEARNERS_QUERY)["subject_code"].unique()

# locate subjects that have more than MAX_NUMBER_OF_WORKOUTS
subjects_to_drop = sessions[
    (~sessions["subject_code"].isin(target_subjects))
    & (~sessions["subject_code"].isin(learner_subjects))
    & ((sessions["weekly_workouts"] > MAX_NUMBER_OF_WORKOUTS) | (sessions["weekly_workouts"].isna()))
]["subject_code"].unique()
valid_learners = sessions[
    (sessions["subject_code"].isin(learner_subjects))
    & (sessions["weekly_workouts"] <= MAX_NUMBER_OF_WORKOUTS)
]["subject_code"].unique()

print(f"Target subject (N): {len(target_subjects)}")
print(f"Valid learners (N): {len(valid_learners)}")
print(f"Subjects to drop (N): {len(subjects_to_drop)}")

Target subject (N): 148
Valid learners (N): 113
Subjects to drop (N): 1322


In [6]:
# sessions[~sessions["subject_code"].isin(target_subjects) & (sessions["subject_code"].isin(learner_subjects)) & (sessions["weekly_workouts"] > MAX_NUMBER_OF_WORKOUTS)].shape
# len(subjects_to_drop)

In [7]:
source = Path("/media/storage/phd//athletes_brain/data/interim")
destination = Path("/media/storage/phd//athletes_brain/data/processed")

# unique_measures = DIFFUSION_MEASURES + STRUCTURAL_MEASURES + CONNECTOME_MEASURES
unique_measures = DIFFUSION_MEASURES + STRUCTURAL_MEASURES


results = {m: {} for m in unique_measures}

for m in unique_measures:
    print(f"Processing {m}")

    data = pd.read_csv(source / f"{m}.csv", index_col=0)
    data["subject_code"] = (
        data["subject_code"]
        .astype(str)
        .str.replace("-", "")
        .str.replace(" ", "")
        .str.replace("_", "")
        .str.zfill(4)
    )
    # data = data.drop_duplicates(subset=["subject_code", "index"], keep="last")
    # tag athletes
    data["target"] = data["subject_code"].isin(target_subjects)
    data["learner"] = data["subject_code"].isin(learner_subjects)
    data["cs_valid"] = True
    data.loc[
        (data["learner"]) & (~data["target"]) & (~data["subject_code"].isin(valid_learners)),
        "cs_valid",
    ] = False
    # break
    # drop learners that have more than one session
    # learners = data[data["condition"].isin(["Learning", "Novice Control"]) & (data["study"] == "Sport")]
    # learners = learners[learners.duplicated(subset=["subject_code"], keep=False)]
    # data = data.drop(learners.index)
    # data = data.drop(
    #     data[
    #         (data["condition"].isin(["Learning", "Novice Control"])) & (data["study"] == "Sport")
    #     ].index
    # )
    # data = data.drop(data[(data["target"]) & (~data["group"].isin(["Climbing","Bjj"]))].index)
    # data = data.drop(
    #     data[(data["exercise_frequency"] > MAX_NUMBER_OF_WORKOUTS) & (~data["target"])].index
    # )
    # drop "subjects_to_drop"
    # data = data[~data["subject_code"].isin(subjects_to_drop)]
    data.loc[data["subject_code"].isin(subjects_to_drop), "cs_valid"] = False
    # data[data["exercise_frequency"] < MAX_NUMBER_OF_WORKOUTS, "target"] = False
    data.to_csv(destination / f"{m}.csv")
    print(f"Found {data['subject_code'].nunique()} subjects with {m} data")

Processing adc


/tmp/ipykernel_4189425/3832128871.py:13: DtypeWarning: Columns (16,23,24,25,26,27,33,39,43,44,45,46,47,49,50,51,52,57,58,59,60,61,62,64,78,80,81,86,87,88,89,96,97,98,108) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(source / f"{m}.csv", index_col=0)


Found 2709 subjects with adc data
Processing fa


/tmp/ipykernel_4189425/3832128871.py:13: DtypeWarning: Columns (16,23,24,25,26,27,33,39,43,44,45,46,47,49,50,51,52,57,58,59,60,61,62,64,78,80,81,86,87,88,89,96,97,98,108) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(source / f"{m}.csv", index_col=0)


Found 2709 subjects with fa data
Processing rd


...

Found 2710 subjects with gm_vol data
Processing wm_vol


/tmp/ipykernel_4189425/3832128871.py:13: DtypeWarning: Columns (1,6,7,8,9,10,11,12,68,74,78,79,80,81,82,84,85,86,87,92,93,94,95,96,97,99,113,115,116,121,122,123,124,125,132,133,134) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(source / f"{m}.csv", index_col=0)


Found 2710 subjects with wm_vol data
Processing csf_vol


/tmp/ipykernel_4189425/3832128871.py:13: DtypeWarning: Columns (1,6,7,8,9,10,11,12,68,74,78,79,80,81,82,84,85,86,87,92,93,94,95,96,97,99,113,115,116,121,122,123,124,125,132,133,134) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(source / f"{m}.csv", index_col=0)


Found 2710 subjects with csf_vol data


In [ ]:
# data["target"] = data["subject_code"].isin(target_subjects)
# data["learner"] = data["subject_code"].isin(learner_subjects)
# data["valid"] = True
# data.loc[data["learner"] & (~data["subject_code"].isin(valid_learners)), "valid"] = False
data.drop_duplicates(subset=["subject_code", "index"], keep="last")["target"].value_counts(normalize=True)
# data["index"].unique()

target
False    0.94825
True     0.05175
Name: proportion, dtype: float64

In [ ]:
# data["target"] = data["subject_code"].isin(target_subjects)
# data["learner"] = data["subject_code"].isin(learner_subjects)
# data["valid"] = True
# data.loc[data["learner"] & (~data["subject_code"].isin(valid_learners)), "valid"] = False
data[data["cs_valid"]].drop_duplicates(subset=["subject_code", "index"], keep="last")["target"].value_counts(normalize=True)
# data["index"].unique()

target
False    0.910323
True     0.089677
Name: proportion, dtype: float64

In [ ]:
d = data.drop_duplicates(subset=["session_id"], keep="first")

In [ ]:
# d[d["learner"]].drop_duplicates(subset="subject_code")["valid"].value_counts()
d.drop_duplicates(subset="subject_code")["cs_valid"].value_counts()

cs_valid
True     1548
False    1135
Name: count, dtype: int64